In [79]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score, LeaveOneOut, cross_val_predict
from sklearn.pipeline import Pipeline, make_pipeline

In [170]:
df = pd.read_csv('../others/data_aim_speed.csv')
df = df[df["y_axis"]>=0]
df.head()

,x_axis,y_axis,x_speed,y_speed
0,0,13,0.009845,0.183703
1,0,14,0.040972,0.839762
2,0,18,0.191212,5.146364
3,0,24,0.379694,14.440056
4,0,31,0.541891,NaN


# Adds data

In [171]:
y2x = 1.444  # sensibility 100/100

dfmerg = df.reset_index().merge(df, left_on=['x_axis', 'y_axis'], right_on=['y_axis', 'x_axis'])
dftmp = dfmerg[dfmerg.isna().any(axis=1)]
mask = (dftmp['y_speed_x'].isna()) & (~dftmp['x_speed_y'].isna())
dftmp.loc[mask, 'y_speed_x'] = dftmp[mask]['x_speed_y'] / y2x

mask = (dftmp['x_speed_x'].isna()) & (~dftmp['y_speed_y'].isna())
dftmp.loc[mask, 'x_speed_x'] = dftmp[mask]['y_speed_y'] * y2x

df.loc[dftmp['index'], ['x_speed', 'y_speed']] = dftmp[['x_speed_x', 'y_speed_x']].values
df

,x_axis,y_axis,x_speed,y_speed
0,0,13,0.009845,1.837033e-01
1,0,14,0.040972,8.397625e-01
2,0,18,0.191212,5.146364e+00
3,0,24,0.379694,1.444006e+01
4,0,31,0.541891,2.813094e+01
...,...,...,...,...
142,126,0,514.629714,1.771429e-02
143,127,0,520.646572,1.000000e-14
144,127,2,520.794088,1.385725e-01
145,127,64,534.092497,9.465610e+01


In [172]:
df2 = df.copy()
df2 = df.rename({'x_axis':'y_axis','y_axis':'x_axis'},axis=1)
new_x_speed = df2['y_speed']*y2x
new_y_speed = df2['x_speed']/y2x
df2['x_speed'] = new_x_speed
df2['y_speed'] = new_y_speed
df = pd.concat((df,df2), ignore_index=True)
df = df.loc[df[['x_axis','y_axis']].drop_duplicates().index]
df

,x_axis,y_axis,x_speed,y_speed
0,0,13,9.844610e-03,0.183703
1,0,14,4.097207e-02,0.839762
2,0,18,1.912121e-01,5.146364
3,0,24,3.796940e-01,14.440056
4,0,31,5.418912e-01,28.130944
...,...,...,...,...
279,0,110,2.925397e-01,293.993189
281,3,120,1.234920e+00,333.398063
282,0,125,4.348530e-02,352.754473
284,0,127,1.444000e-14,360.558568


In [173]:
df[df.isna().any(axis=1)]

,x_axis,y_axis,x_speed,y_speed
8,0,96,0.491205,NaN
13,1,46,2.154924,NaN
14,1,64,2.217958,NaN
15,1,127,0.072241,NaN
17,2,32,2.840182,NaN
22,5,100,5.595488,NaN
24,5,127,0.956895,NaN
33,10,20,7.555886,NaN
34,10,64,17.396694,NaN
35,10,96,13.182239,NaN


In [174]:
df = df[df['x_speed']>1e-7]
# df = df[df['y_speed']>1e-7]

Removing $y_{axis}>x_{axis}$

In [5]:
df = df[df['x_axis']>=df['y_axis']]

In [259]:
from scipy import optimize
from sklearn.linear_model import BayesianRidge


def g(x, y, a1=1, a2=1, c=2):
    return ((a1*x)**c+(a2*y)**c)**(1/c)


def empirical_loss(params, data):
    a, b1, c, d, p1, p2, p3 = params
    x_axis = data['x_axis'].values
    y_axis = data['y_axis'].values
    x_speed = data['x_speed'].values

    r = ((a*(x_axis+b1))**2 + (y_axis+b1)**2)**0.5+d
    r2 = r*r
    r3 = r2*r
    r_out = p3*r3+p2*r2+p1*r
    x_preds = r_out * (x_axis+c) / (r+1e-18)

    loss = ((x_speed/x_preds-1)**2).max()**0.5 + ((x_preds/x_speed-1)**2).max()**0.5
    return loss
    # return ((x_speed-x_preds)**2).mean()


def calc_feats2(params, data, include_axes=False, degree=4):
    a1, a2, b1, d = params
    # a2 = a1 = 1
    b2 = b1

    sc = 64

    b1 = b1/sc
    b2 = b2/sc
    d = d/sc

    x_axis = data['x_axis'].values/sc + b1
    y_axis = data['y_axis'].values/sc + b2

    r = g(x_axis, y_axis, a1, a2, 2)+d
    Rs = [r]
    for i in range(degree-1):
        Rs.append(Rs[-1]*r)
    if(include_axes):
        Rs += [x_axis, y_axis]
    Rs = np.vstack(Rs).T

    return Rs, x_axis


def empirical_loss2(params, data, return_poly_params=False):
    Rs, x_axis = calc_feats2(params, data)
    target_axis = x_axis
    target_speed = data['x_speed'].values
    r = Rs[:, 0]
    target_speed_t = target_speed*r/target_axis

    linreg = BayesianRidge(n_iter=500, tol=1e-4)
    reg = Pipeline([
        # ('scaler', StandardScaler()),
        # ('poly', PolynomialFeatures(2)),
        ('reg', linreg)])
    reg.fit(Rs, target_speed_t, reg__sample_weight=1/(data['x_axis'].values+data['y_axis'].values))
    pparams = linreg.intercept_, *linreg.coef_
    r_out = reg.predict(Rs)
    x_preds = r_out * target_axis / (r+1e-18)

    mx = np.max((target_speed, x_preds), axis=0)
    mn = np.min((target_speed, x_preds), axis=0)
    # loss = (((mx+1e-18)/(mn+1e-18)-1)**2).max()**0.5
    loss = (((mx+1e-18)/(mn+1e-18)-1)**4).mean()**0.25
    if(return_poly_params):
        return loss, pparams
    return loss


def empirical_loss3(params, data, return_poly_params=False):
    Rs, x_axis = calc_feats2(params, data, include_axes=True, degree=1)
    target_axis = x_axis
    target_speed = data['x_speed'].values
    r = Rs[:, 0]
    target_speed_t = target_speed*r/target_axis

    linreg = BayesianRidge(n_iter=500, tol=1e-4)
    reg = Pipeline([
        # ('scaler', StandardScaler()),
        ('poly', PolynomialFeatures(2)),
        ('reg', linreg)])
    reg.fit(Rs, target_speed_t, reg__sample_weight=1/(data['x_axis'].values+data['y_axis'].values))
    pparams = linreg.intercept_, *linreg.coef_
    r_out = reg.predict(Rs)
    x_preds = r_out * target_axis / (r+1e-18)

    # reg = BayesianRidge()
    # reg.fit(Rs, target_speed, sample_weight=1/(data['x_axis'].values+data['y_axis'].values))
    # pparams = reg.intercept_, *reg.coef_
    # x_preds = reg.predict(Rs)

    mx = np.max((target_speed, x_preds), axis=0)
    mn = np.min((target_speed, x_preds), axis=0)
    # loss = (((mx+1e-18)/(mn+1e-18)-1)**2).max()**0.5
    loss = (((mx+1e-18)/(mn+1e-18)-1)**4).mean()**0.25
    if(return_poly_params):
        return loss, pparams
    return loss


dfx = df.drop('y_speed', axis=1).dropna()
# dfx = df.dropna()
# mask = dfx['x_axis'] < 43
mask = dfx['y_axis'] > 32
mask &= dfx['y_axis'] > dfx['x_axis']
dfx = dfx[mask]
print("Number of data points:", len(dfx))

loss_function = empirical_loss3

sol = optimize.minimize(loss_function, [1.0, 1.0, 0.5, -13], args=dfx, method='Nelder-Mead',
                        options={'maxiter': 500, 'xatol': 1e-10, 'disp': False})['x']

params_ranges = (
    slice(0.1, 1.1, 0.1),  # a1
    slice(0.1, 1.1, 0.1),  # a2
    slice(0.2, 1, 0.1),  # b1
    # slice(0.2, 1, 0.1),  # b2
    # slice(0.1, 2.1, 0.1),  # c
    slice(-14, -11, 0.5),  # d
)
# sol = optimize.brute(loss_function, params_ranges, args=(dfx,),
#                      workers=8, full_output=False)  # , finish=None)
loss, other_params = loss_function(sol, dfx, return_poly_params=True)
sol = np.hstack((sol, other_params[:8]))

print('=====')
pparams = ['p%d' % i for i in range(len(other_params))]
# for name, val in zip(['b1', 'd']+pparams, sol):
for name, val in zip(['a1', 'a2', 'b1', 'd']+pparams[:8], sol):
    print("%s:" % name, val)
print("")
print("%.2f%% loss" % (100*loss))

Number of data points: 52
=====
a1: -0.2922793758025273
a2: -0.08581708871785598
b1: 0.4908187236469379
d: 9.836663582354646
p0: -64.8176917605159
p1: 2.9295503165879113e-09
p2: 356.26917170787686
p3: -40.93252149220387
p4: 60.277451426917274
p5: 110.88209860919717
p6: 68.51892707304077
p7: -238.13257607447463

1.52% loss


- x_axis < 43
- x_axis >= 43
- Se x_axis<y_axis, usar limiar y_axis<=32 e y_axis>32

- Usar média de loss elevada ($E[L^4]^{0.25}$) a quarta é melhor do que ($E[L^2]^{0.5}$) e ($\max[L^2]^{0.5}$)

In [123]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone
from scipy import optimize


class MyBinner(BaseEstimator, RegressorMixin):
    def __init__(self, base_regressor: BaseEstimator):
        self.base_regressor = base_regressor

    def _make_masks(self, X):
        mask1 = X['x_axis'] < 43
        mask2 = X['x_axis'] >= 43

        return (mask1, mask2)

    def fit(self, X, y=None, **kwargs):
        masks = self._make_masks(X)
        self.regressors = [clone(self.base_regressor).fit(X[m], y[m]) for m in masks]
        return self

    def predict(self, X):
        masks = self._make_masks(X)

        y = np.empty(len(X), dtype=float)
        for reg, m in zip(self.regressors, masks):
            idxs = np.where(m)[0]
            yi = reg.predict(X[m])
            y[idxs] = yi
        return y


class FinalOWReg(BaseEstimator, RegressorMixin):
    class _BaseReg(BaseEstimator, RegressorMixin):
        def __init__(self, loss_func=empirical_loss2,
                     feat_extractor=calc_feats2,
                     feat_extractor_params={'include_axes': False, 'degree': 4}):
            self.loss_func = loss_func
            self.feat_extractor = feat_extractor
            self.feat_extractor_params = feat_extractor_params

        def fit(self, X, y):
            self.params = optimize.minimize(self.loss_func, [1.0, 1.0, 0.5, -13], args=X, method='Nelder-Mead',
                                            options={'maxiter': 500, 'xatol': 1e-10, 'disp': False})['x']
            Rs, x_axis = self.feat_extractor(self.params, X, **self.feat_extractor_params)
            target_axis = x_axis
            target_speed = y
            r = Rs[:, 0]
            target_speed_t = target_speed*r/target_axis
            self.reg = BayesianRidge(n_iter=500, tol=1e-4)
            self.reg.fit(Rs, target_speed_t, reg__sample_weight=1 /
                         (data['x_axis'].values+data['y_axis'].values))

            return self

        def predict(self, X):
            Rs, x_axis = self.feat_extractor(self.params, X, **self.feat_extractor_params)
            r = Rs[:, 0]
            pred_speed_t = self.reg.predict(Rs)
            pred_speed = pred_speed_t * x_axis / r
            return pred_speed

    @staticmethod
    def _fitreg1(X, y):
        # TODO
        return params, reg

    def fit(self, X, y):

        return self


class BruteReg3(BaseEstimator, RegressorMixin):
    def fit(self, X, y, *args, **kwargs):
        X = X.copy()
        if(not isinstance(X, pd.DataFrame)):
            X = pd.DataFrame(X, columns=['x_axis', 'y_axis'])
        X['x_speed'] = y[:, 0]
        # params = optimize.minimize(empirical_loss2, [1, 0.5, -12], args=X, method='Nelder-Mead',
        #                            options={'maxiter': 10000, 'xatol': 1e-11, 'disp': False})['x']

        # params_ranges = (
        #     slice(1.0, 1.5, 0.1),  # a1
        #     slice(1.0, 1.5, 0.1),  # a2
        #     slice(0.2, 1, 0.05),  # b1
        #     slice(0.2, 1, 0.05),  # b2
        #     # slice(0.1, 2.1, 0.1),  # c
        #     slice(-14, -11, 0.1),  # d
        # )
        # params = optimize.brute(empirical_loss2, params_ranges, args=(X,),
        #                         workers=1, full_output=False)  # , finish=None)

        _, other_params = empirical_loss2(params, X, return_poly_params=True)
        self.params = np.hstack((params, other_params))

        #X2 = X[['x_speed', 'x_axis', 'y_axis']].copy()
        # X2 = X[['x_speed']].copy()
        # X2 = pd.DataFrame()
        # X2['y_speed_ref'] = self.predict(X, predict_y_ref=True)[:, 1]

        # self.x2yspeed_reg = make_pipeline(StandardScaler(), BayesianRidge())
        # self.x2yspeed_reg.fit(X2, y[:, 1])

        return self

    def predict(self, X, predict_y_ref=False):
        # a, b1, c, d, p1, p2, p3, p0 = self.params
        # a1, a2, b1, b2, c, d, p1, p2, p3, p0 = self.params
        a1, a2, b1, b2, d = self.params[:5]
        pparams = self.params[5:]
        # a2 = a1 = 1
        # b2 = b1

        sc = 64
        b1 = b1/sc
        b2 = b2/sc
        d = d/sc

        if(not isinstance(X, pd.DataFrame)):
            X = pd.DataFrame(X, columns=['x_axis', 'y_axis'])
        x_axis = X['x_axis'].values/sc + b1
        y_axis = X['y_axis'].values/sc + b2

        r = ((a1*x_axis)**2 + (a2*y_axis)**2)**0.5 + d
        Rs = [r]
        for i in range(len(pparams)-2):
            Rs.append(Rs[-1]*r)
        Rs = np.vstack(Rs).T
        r_out = (pparams[1:] @ Rs.T) + pparams[0]
        X_preds = r_out * x_axis / r
        Y_preds = r_out * y_axis / r / 1.444
        # if(not predict_y_ref):
        #     # XX = np.stack((X_preds, X['x_axis'].values, X['y_axis'].values,Y_preds)).T
        #     # XX = pd.DataFrame(XX, columns=['x_speed', 'x_axis','y_axis','y_speed_ref'])
        #     XX = np.stack((Y_preds)).T
        #     XX = pd.DataFrame(XX, columns=['y_speed_ref'])
        #     Y_preds = self.x2yspeed_reg.predict(XX)
        return np.stack((X_preds, Y_preds)).T

In [86]:
from scipy import interpolate


class InterReg(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        self.f = interpolate.interp2d(X['x_axis'].values, X['y_axis'].values, y, kind='cubic')
        # self.f = interpolate.bisplrep(X['x_axis'].values, X['y_axis'].values, y)
        return self

    def predict(self, X):
        if(isinstance(X, pd.DataFrame)):
            X = X.values
        Y = np.empty(len(X))
        for i, (x_axis, y_axis) in enumerate(X):
            Y[i] = self.f(x_axis, y_axis)
        return Y


class InterReg1D(BaseEstimator, RegressorMixin):
    def fit(self, X, y):
        R = g(X['x_axis'].values, X['y_axis'].values)
        self.f = interpolate.interp1d(R, y, kind='linear', fill_value='extrapolate')
        return self

    def predict(self, X):
        if(isinstance(X, pd.DataFrame)):
            X = X.values
        R = g(X[:, 0], X[:, 1])
        Y = np.empty(len(X))
        for i, r in enumerate(R):
            Y[i] = self.f(r)
        return Y

In [87]:
# import torch
# from torch import nn
# from skorch import NeuralNetRegressor
# import skorch
# from adabelief_pytorch import AdaBelief

# class MyArch(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(6, 64), nn.Sigmoid(),
#             nn.Linear(64,64), nn.Sigmoid(),
#             nn.Linear(64,1)
#         )
        
#     def forward(self,X):
#         return self.fc(X)
    
# net_params = {
#     'max_epochs':25,
#     'lr':2e-2,
#     'batch_size':8,
#     'iterator_train__shuffle':True,
#     'train_split': None,#skorch.dataset.ValidSplit(32,stratified=False),
# }

# net = NeuralNetRegressor(module=MyArch,**net_params)

In [124]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import AdaBoostRegressor
from mlinsights.mlmodel import PiecewiseRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor

dfx = df.dropna()
# mask = dfx['x_axis'] < 43
# mask &= dfx['y_axis'] >= dfx['x_axis']
# mask = (dfx['x_axis'] < 43) & (dfx['y_axis'] < dfx['x_axis'])
mask = (dfx['y_axis'] < 43) & (dfx['y_axis'] > dfx['x_axis'])
dfx = dfx[mask]
print("data points:", len(dfx))

X = dfx[['x_axis', 'y_axis']].copy()
# X['r_axis'] = g(X['x_axis'],X['y_axis'],c=2)
# for i in range(2, 7):
#     X['x_axis_%d' % i] = X['x_axis']**i
#     X['y_axis_%d' % i] = X['y_axis']**i

# target = (dfx['x_speed']**2+dfx['y_speed']**2)**0.5
target = dfx[['x_speed', 'y_speed']].values

### Regressor 1 ###
# base_reg = Pipeline([
#     ('polyfeats', PolynomialFeatures(6, include_bias=False)),
#     ('scaler', StandardScaler()),
#     ('reg', BayesianRidge())])

# head_reg = Pipeline([('polyfeats', PolynomialFeatures(3, include_bias=False)),
#                      ('scaler', StandardScaler()),
#                      ('reg', LinearRegression())])
# reg = StackingRegressor([('base_reg1', base_reg)],
#                         RandomForestRegressor(min_impurity_decrease=1e-6),
#                         cv=LeaveOneOut(), passthrough=False)
# reg = base_reg

### Regressor 2 ###
reg = BruteReg3()
# head_reg = Pipeline([('polyfeats', PolynomialFeatures(4, include_bias=False)),
#                      ('scaler', StandardScaler()),
#                      ('reg', BayesianRidge())])


# reg = StackingRegressor([('base_reg1', reg)],
#                         final_estimator=MultiOutputRegressor('a'),
#                         cv=LeaveOneOut(), passthrough=False)
###################
### Regressor 3 ###
# reg = PiecewiseRegressor(
#     binner=DecisionTreeRegressor(min_samples_leaf=30),
#     estimator=BruteReg3())
# reg = LinearRegression()
# reg = MyBinner(BruteReg3())
###################
### Regressor 4 ###
# reg = InterReg()
###################


preds = cross_val_predict(reg, X, target, cv=LeaveOneOut(), n_jobs=8)
loss = np.max((abs(target/preds-1), abs(preds/target-1)), axis=0)
R = pd.DataFrame({'loss_x': loss[:, 0], 'loss_y': loss[:, 1],
                  'x_speed_pred': preds[:, 0], 'y_speed_pred': preds[:, 1]})
R = pd.concat((R, dfx.reset_index(drop=True)), axis=1)
if("last_R" in globals()):
    print('last')
    display(last_R[['loss_x', 'loss_y']].agg(['mean', 'std', 'median', 'min', 'max']))
    print()
print('current')
display(R[['loss_x', 'loss_y']].agg(['mean', 'std', 'median', 'min', 'max']))
last_R = R

data points: 57
last


,loss_x,loss_y
mean,0.012524,0.066528
std,0.015059,0.082580
median,0.006963,0.022720
min,0.000090,0.002423
max,0.088614,0.295058



current


,loss_x,loss_y
mean,0.009153,0.099937
std,0.011564,0.054930
median,0.006523,0.095108
min,0.000199,0.007321
max,0.054704,0.254074


In [125]:
# R['r_speed'] = (R['x_speed']**2+R['y_speed']**2)**0.5
R.sort_values('loss_x', ascending=False).head(10)

,loss_x,loss_y,x_speed_pred,y_speed_pred,x_axis,y_axis,x_speed,y_speed
16,0.054704,0.114020,0.158343,0.159066,8,10,0.167005,0.142785
29,0.051670,0.087168,0.076026,1.860680,0,15,0.079955,1.711493
0,0.043008,0.079037,0.009439,0.198223,0,13,0.009845,0.183703
28,0.029404,0.007321,6.607784,5.113474,13,14,6.802081,5.076312
13,0.026974,0.135012,0.038688,0.052545,6,11,0.037672,0.046295
37,0.025902,0.015308,6.906386,6.664375,12,16,7.085273,6.563895
1,0.017388,0.099812,0.040272,0.923581,0,14,0.040972,0.839762
52,0.016743,0.125288,1.904122,34.930393,1,36,1.872765,39.306745
44,0.011418,0.062898,2.239094,17.134327,2,26,2.213816,18.212041
18,0.010959,0.139611,1.102534,1.271788,8,12,1.090582,1.115984


In [13]:
R['y_speed_pred']/R['y_speed']

0     0.999333
1     1.012952
2     1.005944
3     1.019187
4     0.998222
5     0.998767
6     1.008863
7     1.010129
8     1.015031
9     1.012158
10    1.010765
11    1.013650
12    1.013062
13    1.014687
14    1.014585
15    1.013661
16    1.009334
17    1.002600
18    1.002699
19    1.021547
20    1.019808
21    1.030144
22    1.028679
23    1.027556
24    1.026232
25    1.021604
26    1.012563
27    1.048191
28    1.056701
29    1.042782
30    1.066216
31    1.059245
32    1.114688
33    1.086784
34    1.124317
35    1.139714
36    1.121867
37    1.152127
38    1.158850
39    1.172852
40    1.182848
41    1.193496
42    1.208456
43    1.221974
44    1.247268
45    1.260621
46    1.247689
47    1.303545
48    1.317363
49    1.293131
50    1.340315
51    1.045507
52    0.998142
53    0.997444
54    0.997676
55    0.996862
56    0.996920
dtype: float64

# Plotting Dtree

In [308]:
import graphviz
from sklearn import tree

reg = PiecewiseRegressor(binner=DecisionTreeRegressor(min_samples_leaf=7),
                         estimator=BruteReg3())
reg.fit(X,target)
dot_data = tree.export_graphviz(reg.binner_,out_file=None,feature_names=X.columns)
graph = graphviz.Source(dot_data)
graph.render("image",view=True)

'image.pdf'

In [ ]:
def f(x,y):
    x+=1
    y+=1
    r = (x**2+y**2)**0.5
    ro = 5*(r-12.7)
    xo = ro * x/r
    yo = ro * y/r
    return xo,yo

[f(i,0) for i in range(127)]